# State Shapefile Mapping
- To generate random coordinate using Nominatim specified by user, this can be city
- To check the city lies in which state, and thus it will become the region on interest

In [1]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import leafmap.foliumap as leafmap
import numpy as np
#leafmap.update_package()

In [2]:
# read file
df = gpd.read_file("../data/shapefile/polbnda_mys.shp")
df.head(n = len(df))

,f_code,coc,nam,laa,pop,ypc,adm_code,salb,soc,geometry
0,FA001,MYS,KELANTAN,GUA MUSANG,-99999999,0,UNK,UNK,MYS,"POLYGON ((101.58979 4.55280, 101.58880 4.55438..."
1,FA001,MYS,JOHOR,KOTA TINGGI,-99999999,0,UNK,UNK,MYS,"POLYGON ((103.72271 1.66005, 103.72056 1.66298..."
2,FA001,MYS,JOHOR,SEGAMAT,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70207 2.83330, 102.71049 2.83064..."
3,FA001,MYS,PAHANG,BERA,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70207 2.83330, 102.70186 2.83363..."
4,FA001,MYS,PAHANG,ROMPIN,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70289 3.31518, 102.70296 3.31524..."
...,...,...,...,...,...,...,...,...,...,...
231,FA001,MYS,SABAH,KUDAT,-99999999,0,UNK,UNK,MYS,"POLYGON ((117.33995 7.27316, 117.34082 7.27270..."
232,FA001,MYS,SABAH,SEMPORNA,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.78781 4.58668, 118.78745 4.58640..."
233,FA001,MYS,SABAH,SEMPORNA,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.76195 4.41550, 118.76195 4.41550..."
234,FA001,MYS,SABAH,SANDAKAN,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.10109 5.90575, 118.10128 5.90518..."


- Using intersection idea to check whether a point lies in which district
- Then derive the shape and district and state name

# Intersection Checking

In [3]:
# testing geolocator library
geolocator = Nominatim(user_agent="u2004763@siswa.um.edu.my")
location = geolocator.geocode("Johor Bahru")
point = location.point

# make the output as a dataframe
df_point = pd.DataFrame({'City': ['Johor Bahru'], 'Latitude': [point.latitude], 'Longitude': [point.longitude]})

# make the output as a geodataframe
gdf_point = gpd.GeoDataFrame(df_point, geometry=gpd.points_from_xy(df_point.Longitude, df_point.Latitude))

# preview
gdf_point.head(n = len(gdf_point))

,City,Latitude,Longitude,geometry
0,Johor Bahru,1.503555,103.749559,POINT (103.74956 1.50355)


In [4]:
for index in range(len(df)):
    if df['geometry'][index].contains(gdf_point['geometry'][0]) == True:
        print(df['nam'][index])
        print(df['laa'][index])
        print(df['geometry'][index])
        break

JOHOR
JOHOR BAHRU
POLYGON ((103.72271424360041 1.6600463188421426, 103.72695108351576 1.6625851658497277, 103.73073403853985 1.6648227409519298, 103.73377562961278 1.6666064155320983, 103.73695615662191 1.6683759578073392, 103.73960665070057 1.669862641689362, 103.74109859755356 1.670833734776731, 103.74436555614913 1.6704363932799553, 103.74880226202214 1.6699213571358575, 103.75244570919983 1.6694908584074604, 103.75765720807223 1.6687465868034004, 103.7607264312963 1.668218824537919, 103.7636751510277 1.6678453294705093, 103.7660142102536 1.6674923179724332, 103.7660560483471 1.6666553924280831, 103.7661718859227 1.663950152868213, 103.76617989282815 1.6611774984848635, 103.76638457137646 1.658241234664747, 103.76641184020431 1.6570922680377933, 103.76734999619875 1.6574096922009778, 103.76814925565576 1.6577502822181327, 103.76856270462721 1.6580245556129405, 103.76885117647846 1.6584530998203126, 103.7692522127279 1.6590168984564315, 103.7698010044949 1.6595258227137333, 103.77010

In [5]:
df = df.loc[index,:].to_frame().transpose()
df = df[['nam', 'laa', 'geometry']]
df = gpd.GeoDataFrame(df, geometry=df.geometry, crs="EPSG:4326")
df.reset_index()

,index,nam,laa,geometry
0,23,JOHOR,JOHOR BAHRU,"POLYGON ((103.72271 1.66005, 103.72695 1.66259..."


In [6]:
from shapely.geometry import Polygon, MultiPolygon
df['geometry'] = MultiPolygon([df['geometry'].to_numpy()[0]])

In [7]:
df

,nam,laa,geometry
23,JOHOR,JOHOR BAHRU,"MULTIPOLYGON (((103.72271 1.66005, 103.72695 1..."


# Mapping

- Fill color not working, resolution https://github.com/opengeos/leafmap/discussions/527

In [9]:
m = leafmap.Map(center=[df_point['Latitude'][0], df_point['Longitude'][0]], zoom=15, google_map="HYBRID")

style = {
    "stroke": True,
    "color": "#0000ff",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.1,
}

m.add_points_from_xy(
    gdf_point,
    x="Longitude", 
    y="Latitude",
    icon_names=['gear', 'map', 'leaf', 'globe'],
)
m.add_gdf(df, layer_name="Johor Bahru", style=style)
m